In [ ]:
%tensorflow_version 2.x

In [ ]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Dropout
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
ImageDataGenerator.flow_from_directory?

In [ ]:
# !wget https://www.dropbox.com/s/uo4a18fhn7sl2d9/cat_dog_sub.zip

In [ ]:
# !unzip cat_dog_sub.zip 

In [ ]:
# !rm -rf __MACOSX

In [ ]:
base_model=MobileNet(weights='imagenet',include_top=False)

In [ ]:
# base_model.summary()

In [ ]:
CLASSES = 2
x = base_model.output
x = GlobalAveragePooling2D(name='avg_pool')(x)
x = Dropout(0.4)(x)
predictions = Dense(CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False
      
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# model.summary()

In [ ]:
os.listdir('cat_dog_sub')

In [ ]:
# !rm -rf /content/cat_dog_sub/.DS_Store

In [ ]:
WIDTH = 299
HEIGHT = 299
BATCH_SIZE = 32
TRAIN_DIR=r'/content/cat_dog_sub/train'
TEST_DIR=r'/content/cat_dog_sub/test'
# data prep
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(HEIGHT, WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='categorical')
    
validation_generator = validation_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(HEIGHT, WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

In [ ]:
data=next(train_generator)

In [ ]:
data[1].shape

In [ ]:
import tensorflow as tf

In [ ]:
EPOCHS = 5
BATCH_SIZE = 32
STEPS_PER_EPOCH = train_generator.n//train_generator.batch_size
VALIDATION_STEPS=validation_generator.n//train_generator.batch_size

In [ ]:
STEPS_PER_EPOCH,VALIDATION_STEPS

In [ ]:
model.fit(
    train_generator,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=validation_generator,
    validation_steps=VALIDATION_STEPS,verbose=1)

In [ ]:
from tensorflow.keras.preprocessing import image

In [ ]:
img = image.load_img(r'cat_dog_sub/test/dog/182.jpg', target_size=(299,299))

In [ ]:
x = image.img_to_array(img)
x = preprocess_input(x)
x = np.expand_dims(x, axis=0)
pred = model.predict(x)[0]

In [ ]:
pred

In [ ]:
train_generator.class_indices